In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

train_sales = pd.read_csv('sales_train_evaluation.csv')
calendar = pd.read_csv('calendar.csv')

In [ ]:
#To reduce memory usage
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

#Reduce memory usage and compare with the previous one to be sure
train_sales = downcast_dtypes(train_sales)

In [ ]:
train_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


Preprocess dataset

In [ ]:
# Preprocess: remove id, item_id, dept_id, cat_id, store_id, state_id columns
startDay = 350  # Remove the first 350 days in train sales data due to zero_inflated data
train_sales = train_sales.T
train_sales = train_sales[6 + startDay:]
train_sales.head(5)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,...,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,...,0,5,4,0,8,0,1,2,0,0
d_353,0,0,0,4,2,0,0,2,1,1,...,0,15,2,0,3,0,1,2,0,0
d_354,0,1,0,2,0,0,0,7,1,0,...,0,5,1,0,3,0,0,0,0,0
d_355,0,0,0,1,2,0,0,0,0,0,...,0,7,1,0,1,0,1,1,0,0


Additional features: event 1, event 2 and SNAP
Besides using sales as a time series feature, festive or sports events can have a strong positive influence on the sales. For example, shoppers are more likely to purchase more snacks or food a day before Thanksgiving or Superbowl. Similarly, we expect more sales during the days with SNAP program. Hence, we inserted 5 additional features: event 1, event 2, SNAP_CA, SNAP_WI, SNAP_TX. The days of SNAP program is different for each store location and therefore we need 3 separate features for SNAP.

In [ ]:
# Initialize a dataframe with zeros for 1969 days in the calendar

daysBeforeEvent1 = pd.DataFrame(np.zeros((1969,1)))
daysBeforeEvent2 = pd.DataFrame(np.zeros((1969,1)))

snap_CA = pd.DataFrame(np.zeros((1969,1)))
snap_TX = pd.DataFrame(np.zeros((1969,1)))
snap_WI = pd.DataFrame(np.zeros((1969,1)))


# Label 1 to one day before the event_name_1 
# Label 1 to one day before the event_name_2
# Sales are likely to increase one day before events like superbowl etc.

# Label 1 to days on snap_CA
# Label 1 to days on snap_TX
# Label 1 to days on snap_WI

for x,y in calendar.iterrows():
    if((pd.isnull(calendar["event_name_1"][x])) == False):
           daysBeforeEvent1[0][x-1] = 1 
            
    if((pd.isnull(calendar["event_name_2"][x])) == False):
           daysBeforeEvent2[0][x-1] = 1    
    
    
    if((pd.isnull(calendar["snap_CA"][x])) == False):
           snap_CA[0][x] = 1    
        
    if((pd.isnull(calendar["snap_TX"][x])) == False):
           snap_TX[0][x] = 1    
        
    if((pd.isnull(calendar["snap_WI"][x])) == False):
           snap_WI[0][x] = 1

Creating training, validation, evaluation dataset
Training Dataset

To train a timeseries dataset, we will use the training dataset, which comprises of features (sales, event 1, event 2, SNAP) from day 350 to 1912.

From this training dataset, we will predict the future sales of each product for 56 days in 2 week windows (predict day 1913 to 1940, and predict day 1941 to 1969).

Validation Dataset

The dataset with the features (sales, event 1, event 2, SNAP) from day 1913 to 1940 is used to validate the predicted values from trained dataset.
Evaluation Dataset

The dataset with the features (sales, event 1, event 2, SNAP) from day 1941 to 1969 is used to evaluate the predicted values from trained dataset.

In [ ]:
# split dataset into evaluation (last 2 weeks), validation (first 2 weeks), training  
# input for predicting validation period day 1941 to 1969

daysBeforeEvent1_eval = daysBeforeEvent1[1941:]
daysBeforeEvent2_eval = daysBeforeEvent2[1941:]

snap_CA_eval = snap_CA[1941:]
snap_TX_eval = snap_TX[1941:]
snap_WI_eval = snap_WI[1941:]


# input for predicting validation period day 1913 to 1941

daysBeforeEvent1_valid = daysBeforeEvent1[1913:1941] 
daysBeforeEvent2_valid = daysBeforeEvent2[1913:1941]

snap_CA_valid = snap_CA[1913:1941] 
snap_TX_valid = snap_TX[1913:1941]
snap_WI_valid = snap_WI[1913:1941]

# input for training as a feature
# daysBeforeEvent1 = daysBeforeEvent1[startDay:1913] 
# daysBeforeEvent2 = daysBeforeEvent2[startDay:1913] 
daysBeforeEvent1 = daysBeforeEvent1[startDay:1941] 
daysBeforeEvent2 = daysBeforeEvent2[startDay:1941]

snap_CA = snap_CA[startDay:1941] 
snap_TX = snap_TX[startDay:1941] 
snap_WI = snap_WI[startDay:1941] 

In [ ]:
#Before concatanation with our main data "dt", indexes are made same and column name is changed to "oneDayBeforeEvent"
daysBeforeEvent1.columns = ["oneDayBeforeEvent1"]
daysBeforeEvent1.index = train_sales.index

daysBeforeEvent2.columns = ["oneDayBeforeEvent2"]
daysBeforeEvent2.index = train_sales.index


snap_CA.columns = ["snap_CA"]
snap_CA.index = train_sales.index

snap_TX.columns = ["snap_TX"]
snap_TX.index = train_sales.index

snap_WI.columns = ["snap_WI"]
snap_WI.index = train_sales.index


In [ ]:
train_sales = pd.concat([train_sales, daysBeforeEvent1, daysBeforeEvent2,
                        snap_CA, snap_TX, snap_WI], axis = 1, sort=False)
train_sales.head()  # additional features (event1, event2, SNAP) are added

,0,1,2,3,4,5,6,7,8,9,...,30485,30486,30487,30488,30489,oneDayBeforeEvent1,oneDayBeforeEvent2,snap_CA,snap_TX,snap_WI
d_351,0,0,0,2,0,0,0,24,3,2,...,0,0,1,0,0,0.0,0.0,1.0,1.0,1.0
d_352,0,0,0,0,0,0,0,9,0,2,...,0,1,2,0,0,1.0,0.0,1.0,1.0,1.0
d_353,0,0,0,4,2,0,0,2,1,1,...,0,1,2,0,0,0.0,0.0,1.0,1.0,1.0
d_354,0,1,0,2,0,0,0,7,1,0,...,0,0,0,0,0,0.0,0.0,1.0,1.0,1.0
d_355,0,0,0,1,2,0,0,0,0,0,...,0,1,1,0,0,0.0,0.0,1.0,1.0,1.0


Standardizing features
It is also important to scale our features across the columns. Each columns represents the sales values of a particular day. This helps to ensure that the sales values are between 0 and 1, and this helps the gradient descent optimization algorithm in LSTM model.

In [ ]:
#Feature Scaling: Scale features using min-max scaler in range 0-1
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range = (0, 1))

train_sales_scaled = sc.fit_transform(train_sales)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
timesteps = 28  # use the last 28 days to predict the next day's sales
X_train = []
y_train = []

for i in range(timesteps, 1941 - startDay):
    X_train.append(train_sales_scaled[i-timesteps:i])
    y_train.append(train_sales_scaled[i][0:30490])

In [ ]:
#Convert to np array to be able to feed the LSTM model
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(1563, 28, 30495)
(1563, 30490)


CNN-LSTM Model
The model used is CNN-LSTM, with regularization parameters such as batch normalization. There are 2 units of CNN (conv1d followed by max pooling layer). The conv1d layer helps to analyse and extract the patterns of the sales in 1 week window (just like extracting "edges" patterns in a typcial conv2d on an image). The max pooling layer then summarizes these patterns into broader, generalizable patterns, which seeks to remove noises in the spikes or troughs in daily sales.

The outputs from CNN is then fed into a LSTM model with 3 layers of bidirectional LSTM. We use an inverted number of LSTM units from 512 to 256 to 128. This has the property of reducing and summarizing the sales patterns for better prediction. The batch normalization helps to improve the robustness of the model by reducing the likelihood of overfitting.

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout

tf.random.set_seed(51)
np.random.seed(51)

n_timesteps = X_train.shape[1]
n_products = X_train.shape[2]


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=128, kernel_size=7,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=(n_timesteps, n_products)),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Conv1D(filters=64, kernel_size=7, 
                           strides=1, activation='relu', padding="causal"),
    tf.keras.layers.MaxPooling1D(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(30490)
])


opt_adam = tf.keras.optimizers.Adam(clipvalue=0.5)

model.compile(loss='mse',
              optimizer=opt_adam, 
              metrics=[tf.keras.metrics.RootMeanSquaredError()])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 28, 128)           27323648  
                                                                 
 max_pooling1d (MaxPooling1D  (None, 14, 128)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 14, 64)            57408     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 7, 64)            0         
 1D)                                                             
                                                                 
 bidirectional (Bidirectiona  (None, 7, 1024)          2363392   
 l)                                                              
                                                        

Callbacks
Callbacks are inserted to monitor the performance of model after 100 epochs. If the performance of the model degrades, then the model is stopped.

In [ ]:
# Define a Callback class that stops training when the val_loss degrades above 150 epochs
# and also saves the model as checkpoints

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)

In [14]:
# Fitting the RNN to the Training set
epochs = 150
batch_size = 100
model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size)

Epoch 1/150
16/16 [==============================] - 70s 4s/step - loss: 0.0276 - root_mean_squared_error: 0.1661
Epoch 2/150
16/16 [==============================] - 56s 4s/step - loss: 0.0209 - root_mean_squared_error: 0.1447
Epoch 3/150
16/16 [==============================] - 58s 4s/step - loss: 0.0184 - root_mean_squared_error: 0.1355
Epoch 4/150
16/16 [==============================] - 57s 4s/step - loss: 0.0159 - root_mean_squared_error: 0.1262
Epoch 5/150
16/16 [==============================] - 55s 3s/step - loss: 0.0148 - root_mean_squared_error: 0.1217
Epoch 6/150
16/16 [==============================] - 56s 4s/step - loss: 0.0146 - root_mean_squared_error: 0.1207
Epoch 7/150
16/16 [==============================] - 57s 4s/step - loss: 0.0144 - root_mean_squared_error: 0.1201
Epoch 8/150
16/16 [==============================] - 56s 4s/step - loss: 0.0143 - root_mean_squared_error: 0.1197
Epoch 9/150
16/16 [==============================] - 57s 4s/step - loss: 0.0142 - root_m

In [15]:
inputs_eval = train_sales[-timesteps:]
inputs_eval = sc.transform(inputs_eval)

inputs = train_sales[-timesteps*2:-timesteps]
inputs = sc.transform(inputs)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


Predicting the future sales for validation and evaluation periods

In [16]:
X_test = []
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
predictions = []

for j in range(timesteps,timesteps + 28):
    predicted_stock_price = model.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30495))
    
    testInput = np.column_stack((np.array(predicted_stock_price),
                                 daysBeforeEvent1_valid.loc[1913 + j - timesteps],
                                 daysBeforeEvent2_valid.loc[1913 + j - timesteps],
                                 snap_CA_valid.loc[1913 + j - timesteps],
                                snap_TX_valid.loc[1913 + j - timesteps],
                                snap_WI_valid.loc[1913 + j - timesteps]))

    X_test = np.append(X_test, testInput).reshape(1,j + 1,30495)
    predicted_stock_price = sc.inverse_transform(testInput)[:,0:30490]
    predictions.append(predicted_stock_price)

In [18]:
X_eval = []
X_eval.append(inputs_eval[0:timesteps])
X_eval = np.array(X_eval)
predictions_eval = []

for j in range(timesteps,timesteps + 28):
    predicted_stock_price = model.predict(X_eval[0,j - timesteps:j].reshape(1, timesteps, 30495))
    
    testInput = np.column_stack((np.array(predicted_stock_price),
                                 daysBeforeEvent1_eval.loc[1941 + j - timesteps],
                                 daysBeforeEvent2_eval.loc[1941 + j - timesteps],
                                 snap_CA_eval.loc[1941 + j - timesteps],
                                snap_TX_eval.loc[1941 + j - timesteps],
                                snap_WI_eval.loc[1941 + j - timesteps]))

    X_eval = np.append(X_eval, testInput).reshape(1,j + 1,30495)
    predicted_stock_price = sc.inverse_transform(testInput)[:,0:30490]
    predictions_eval.append(predicted_stock_price)

Post processing of future sales
For sales with predicted negative values, they are converted into zeros.

In [20]:
import time

submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))
submission = submission.T

submission_eval = pd.DataFrame(data=np.array(predictions_eval).reshape(28,30490))
submission_eval = submission_eval.T

submission = pd.concat((submission, submission_eval), ignore_index=True)

sample_submission = pd.read_csv("sample_submission.csv")
    
idColumn = sample_submission[["id"]]
    
submission[["id"]] = idColumn  

cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]

colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]

submission.columns = colsdeneme

currentDateTime = time.strftime("%d%m%Y_%H%M%S")

cols = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']

submission[cols] = submission[cols].mask(submission[cols] < 0, 0)
submission[cols].to_csv("submission.csv", index=False)

In [21]:
import pickle

In [22]:
submission.to_pickle('submission.pkl')

In [23]:
import pandas as pd
data = pd.read_pickle('submission.pkl')

In [24]:
data

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.807098,0.799738,0.981258,0.860267,0.000000,2.882693,4.815520,0.262061,0.140914,...,1.284472,5.746913,0.000000,0.296132,1.009408,1.255228,1.515092,2.398091,0.000000,0.968898
1,HOBBIES_1_002_CA_1_validation,0.000000,0.120000,0.195552,0.250336,0.000000,0.000000,0.165795,0.000000,0.067401,...,0.000000,0.808950,3.683137,0.373910,0.397177,0.317990,0.005601,0.145442,0.067755,0.036965
2,HOBBIES_1_003_CA_1_validation,0.623618,0.367472,0.221009,0.510167,0.878864,2.795125,0.840954,0.449846,0.000000,...,1.296013,0.000000,2.947885,0.323284,0.194133,0.530042,0.926161,1.385527,0.000000,0.889182
3,HOBBIES_1_004_CA_1_validation,2.353488,2.396892,2.223498,1.875467,3.391804,0.000000,7.610034,1.571221,0.000000,...,0.000000,0.999895,0.000000,1.017512,1.638302,1.393294,0.521501,0.000000,0.866482,7.290441
4,HOBBIES_1_005_CA_1_validation,0.511946,1.519276,1.328810,1.123993,1.672541,0.000000,3.791093,1.865155,2.728486,...,1.578273,0.053242,0.000000,1.342455,1.601980,1.219330,0.573632,1.237733,0.000000,0.036579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.133823,0.117358,0.420184,0.485529,0.322470,0.036295,0.752956,0.559010,0.630486,...,0.715556,0.732360,1.441950,0.353003,0.309406,0.352955,0.492202,0.276530,0.580995,0.861825
60976,FOODS_3_824_WI_3_evaluation,0.307503,0.320254,0.423800,0.404358,0.568445,0.575007,0.036119,0.307783,0.515485,...,0.436726,0.576172,0.225452,0.301047,0.388759,0.410545,0.410653,0.523735,0.658310,0.178666
60977,FOODS_3_825_WI_3_evaluation,0.645764,0.506523,0.752616,0.565281,0.581467,0.181602,0.685664,0.855838,1.354621,...,0.869215,0.438357,0.649221,0.809135,0.855437,0.801032,0.721665,0.804387,0.000000,0.498145
60978,FOODS_3_826_WI_3_evaluation,1.501539,1.356868,1.155162,1.187457,1.095011,1.163144,1.490891,0.883650,0.885568,...,1.236306,1.353837,1.165055,1.878072,1.600353,1.365008,1.151671,0.981411,1.097199,1.546432
